In [5]:
from copy import copy
from datetime import datetime

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN

#torch.manual_seed(0)

In [7]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=216,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=216,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [13]:
def train_models(params, model):
    experiment_note = ''
    path = ''
    for key in params:
        experiment_note += key +'_'+ params[key]+'\n'
        path +=  '_'+ params[key]
    

    logger = Logger(path, experiment_note)
    
    print(logger.path)
    print(experiment_note)

    lr = 0.001

    optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
    criterion = nn.CrossEntropyLoss()
    evo_optim = CrossN()

    def validation(net, dataloader):
        correct = 0
        total = 0
        with torch.no_grad():
            for data in dataloader:
                images, labels = data
                images = images.cuda()
                labels = labels.cuda()
                outputs = net(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        return float(100 * correct / total)


    solver = Solver(
        net,
        optimizer,
        logger,
        criterion,
        validation,
        evo_optim, 
        trainloader,
        testloader,
        testloader,  
        epochs=100,
        evo_step=evo_step,
        child_count=20,
        best_child_count=3,
        mode=mode,
        debug=False,
        lr=lr)

    logger.add_post_result(f'start: {datetime.now()}')
    solver.start()
    logger.add_post_result(f'finish: {datetime.now()}')
    torch.save(net.state_dict(), logger.path + '/model_last.chk')
    logger.close()
    
    

def train_three_types(model):
    models = [copy(model),copy(model),copy(model)]
    evo_step = 10
    modes = ['evo_cross', 'evo_only', 'gradient']

    for mode, model in zip(modes, models): 
        
        params = {'net_name':'resnet',
             'preptrained':'T',
             'mode':mode,
             'evo_step':str(evo_step)}
        
        train_models(params, model)
        model = None
    print('Finished')

In [15]:
net = torchvision.models.mobilenet_v2(pretrained=True)
num_ftrs = net.last_channel

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net.classifier = nn.Linear(num_ftrs, len(classes))
net.cuda()
print()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /home/jovyan/.cache/torch/checkpoints/mobilenet_v2-b0353104.pth
100.0%


AssertionError: 
Found no NVIDIA driver on your system. Please check that you
have an NVIDIA GPU and installed a driver from
http://www.nvidia.com/Download/index.aspx

In [70]:
net = torchvision.models.resnet101(pretrained=True)
net.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [14]:
# num_ftrs = 1000

# classes = ('plane', 'car', 'bird', 'cat',
#         'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# net.classifier = nn.Linear(num_ftrs, len(classes))
# net.cuda()
# print()

mobilenet from F:
- Evo + Cross  - processing
- Evo Only 
- Standard 

resnet from F:
- Evo + Cross 
- Evo Only 
- Standard 

mobilenet pretrained T:
- Evo + Cross  
- Evo Only   
- Standard 

resnet pretrained T:
- Evo + Cross  
- Evo Only 
- Standard 